In [1]:
# import json

# def parse_to_json(data):
#     try:
#         # Use the json.dumps() function to convert Python data to a JSON string.
#         json_data = json.dumps(data, indent=4)
#         return json_data
#     except Exception as e:
#         return str(e)

# # Example usage: #You can add the whole dataset here
# data = {"news": "(Oct 5, 2011  12:11 PM CDT) Police are still hunting for an  armed and dangerous  man who shot and killed three people and wounded five others at the California quarry where he worked this morning. He also shot and injured another woman in an attempted carjacking later. The suspect is identified as 45-year-old Shareef Allman. The San Jose Mercury News says he moonlighted as producer of a show called Real 2 Real on something called CreaTV in the area. (He interviews Jesse Jackson in one YouTube clip, which is in the gallery. Allman, who spread a message of non-violence, also has written a book called Amazing Grace about female victims of domestic violence, notes the Mercury News. The shooting spree occurred about 4:30am local time when a man left a safety meeting at the Lehigh Southwest Cement Permanente Plant, then returned with a rifle and a handgun, according to police. He fled on foot, and a massive manhunt is under way in the San Jose area. Schools are on lockdown, notes AP.", "masked headline": "____rd Victim Dead in Quarry Shooting; Manhunt Still On", "calculation": "Trans(three)", "ans": "3"}, {"news": "(Mar 4, 2014  11:30 AM) The New York Times followed the Best Picture win for 12 Years a Slave by tweeting a link to an 1853 article on Solomon Northup, the film's historical subject. The original article's lead describes what is to follow as  a more complete and authentic record than has yet appeared  on Northup's kidnapping. Authentic, perhaps, but erroneous: It spelled Northup's name wrong in two different ways. A Twitter user caught the errors, and now the Times has corrected them\u2014just 161 years late, Poynter reports. The article spelled Northup's last name as  Northrop,  while the headline spelled it  Northrup.", "masked headline": "NYT Corrects 1853 Piece After ____ Years Win", "calculation": "Copy(12)", "ans": "12"}


In [2]:
# json_data = parse_to_json(data)
# print(json_data)

In [3]:
!pip install transformers

In [4]:
!pip install sentencepiece

In [5]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [6]:
from transformers import T5Tokenizer

In [7]:
import json
import torch
from transformers import Trainer, TrainingArguments

In [8]:
!pip install datasets

In [9]:
from datasets import Dataset

In [10]:
# Load the T5 tokenizer and model
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [11]:
!pip install --quiet pytorch-lightning

In [12]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

In [13]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [14]:
# Load your JSON dataset
with open('/content/Train_Headline_Generation.json', 'r') as json_file:
    #train_dataset = json.load(json_file)
    train_dataset=pd.read_json(json_file)

# train_dataset=pd.read_json('/content/Train_Headline_Generation.json')

In [15]:
# Load your dev JSON dataset
with open('/content/Dev_Headline_Generation.json', 'r') as json_file:
    # dev_dataset = json.load(json_file)
    dev_dataset=pd.read_json(json_file)

In [16]:
train_dataset["news"].str.len().max(), train_dataset["headline"].str.len().max()

(3023, 127)

In [17]:
train_dataset.head()

,news,headline
0,"(Oct 7, 2014 12:40 PM CDT) As of Jan. 1, Walm...",30K Walmart Part-Timers to Lose Health Insurance
1,"(Oct 29, 2013 8:15 AM CDT) Dax Shepard and Kr...",Dax Shepard: Wedding to Kristen Bell Cost $142
2,"(Mar 6, 2016 10:50 AM) Nancy Reagan, the help...",Nancy Reagan Dead at 94
3,"(Aug 15, 2008 5:11 AM CDT) American Airlines ...",American Airlines Faces $7M Fine for Safety Vi...
4,"(Apr 18, 2016 1:02 PM CDT) Ingrid Lyne, the S...",$222K Raised for Kids of Mom Dismembered on Date


In [18]:
dev_dataset.head()

,news,headline
0,"(Oct 5, 2011 12:11 PM CDT) Police are still h...",3rd Victim Dead in Quarry Shooting; Manhunt St...
1,"(Nov 4, 2008 3:19 PM) Stocks rallied on Elect...",Stocks Up 305 in Election Rally
2,"(Dec 24, 2014 11:19 AM) Turns out you won't e...",You Can Watch The Interview at 1pm
3,"(Oct 16, 2014 3:02 AM CDT) Tristen Kurilla, t...","Murder Suspect, 10, Will Stay in Adult Jail"
4,"(Oct 3, 2011 9:10 AM CDT) Tobacco companies w...",Tobacco Firms Knew of Radiation in Cigs—in 1959


In [19]:
x=dev_dataset.iloc[0]
x["news"]

'(Oct 5, 2011  12:11 PM CDT) Police are still hunting for an  armed and dangerous  man who shot and killed three people and wounded five others at the California quarry where he worked this morning. He also shot and injured another woman in an attempted carjacking later. The suspect is identified as 45-year-old Shareef Allman. The San Jose Mercury News says he moonlighted as producer of a show called Real 2 Real on something called CreaTV in the area. (He interviews Jesse Jackson in one YouTube clip, which is in the gallery. Allman, who spread a message of non-violence, also has written a book called Amazing Grace about female victims of domestic violence, notes the Mercury News. The shooting spree occurred about 4:30am local time when a man left a safety meeting at the Lehigh Southwest Cement Permanente Plant, then returned with a rifle and a handgun, according to police. He fled on foot, and a massive manhunt is under way in the San Jose area. Schools are on lockdown, notes AP.'

In [20]:
import re

def remove_timestamp_from_article(article):
    # Define a regular expression pattern to match the timestamp at the beginning
    timestamp_pattern = r'^\([^)]+\)\s*'

    # Use re.sub to replace the timestamp with an empty string
    cleaned_article = re.sub(timestamp_pattern, '', article)

    return cleaned_article

In [21]:
# Assuming train_dataset and dev_dataset are your DataFrames
train_dataset["news"] = train_dataset["news"].apply(remove_timestamp_from_article)
dev_dataset["news"] = dev_dataset["news"].apply(remove_timestamp_from_article)

In [22]:
train_dataset.head()

,news,headline
0,"As of Jan. 1, Walmart will no longer offer 30,...",30K Walmart Part-Timers to Lose Health Insurance
1,Dax Shepard and Kristen Bell got married at th...,Dax Shepard: Wedding to Kristen Bell Cost $142
2,"Nancy Reagan, the helpmate, backstage adviser,...",Nancy Reagan Dead at 94
3,American Airlines faces FAA fines of more than...,American Airlines Faces $7M Fine for Safety Vi...
4,"Ingrid Lyne, the Seattle mom allegedly murdere...",$222K Raised for Kids of Mom Dismembered on Date


In [23]:
dev_dataset.head()

,news,headline
0,Police are still hunting for an armed and dan...,3rd Victim Dead in Quarry Shooting; Manhunt St...
1,Stocks rallied on Election Day as investors ap...,Stocks Up 305 in Election Rally
2,Turns out you won't even have to leave your ho...,You Can Watch The Interview at 1pm
3,"Tristen Kurilla, the Pennsylvania 10-year-old ...","Murder Suspect, 10, Will Stay in Adult Jail"
4,Tobacco companies were well aware that their p...,Tobacco Firms Knew of Radiation in Cigs—in 1959


In [24]:
import pandas as pd
class NewsHeadlineDataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        article_max_token_len: int = 512,
        headline_max_token_len: int = 128
    ):

        self.tokenizer = tokenizer
        self.data = data
        self.article_max_token_len = article_max_token_len
        self.headline_max_token_len = headline_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        Zero_shot_prompt="Look at the following news article and generate headline: "
        article = Zero_shot_prompt + data_row["news"]
        # here I can add prompt like "Look at the news and generate headline" before the news.
        article_encoding = tokenizer(
            article,
            max_length=self.article_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        headline_encoding = tokenizer(
            data_row["headline"],
            max_length=self.headline_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        labels = headline_encoding["input_ids"]
        # giving padding (encoded with 0s) value of -100 so that it doesn't get counted in loss func, from docs
        labels[labels == 0] = -100

        return dict(
            article=article,
            headline=data_row["headline"],
            article_input_ids=article_encoding["input_ids"].flatten(),
            article_attention_mask=article_encoding["attention_mask"].flatten(),
            labels=labels.flatten(),
            labels_attention_mask=headline_encoding["attention_mask"].flatten()
        )

In [27]:
class NewsHeadlineDataModule(pl.LightningDataModule):

    def __init__(
        self,
        train_dataset: pd.DataFrame,
        dev_dataset: pd.DataFrame,
        # val_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 8,
        news_max_token_len: int = 512,
        headline_max_token_len: int = 128
    ):

        super().__init__()

        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        # self.val_df = val_df

        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.news_max_token_len = news_max_token_len
        self.headline_max_token_len = headline_max_token_len

    def setup(self, stage=None):
        self.train_dataset = NewsHeadlineDataset(
            self.train_dataset,
            self.tokenizer,
            self.news_max_token_len,
            self.headline_max_token_len
        )

        self.dev_dataset = NewsHeadlineDataset(
            self.dev_dataset,
            self.tokenizer,
            self.news_max_token_len,
            self.headline_max_token_len
        )

        # self.val_dataset = NewsHeadlineDataset(
        #     self.val_df,
        #     self.tokenizer,
        #     self.article_max_token_len,
        #     self.headline_max_token_len
        # )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True, # used for training
            num_workers=2
        )

    def dev_dataloader(self):
        return DataLoader(
            self.dev_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )

    # def test_dataloader(self):
    #     return DataLoader(
    #         self.val_dataset,
    #         batch_size=self.batch_size,
    #         shuffle=False,
    #         num_workers=2
    #     )

In [28]:
#N_EPOCHS = 10
BATCH_SIZE = 8

data_module = NewsHeadlineDataModule(train_dataset, dev_dataset, tokenizer, batch_size=BATCH_SIZE)

In [29]:
class NewsHeadlineModel(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(model_name, return_dict=True)
        self.val_loss = []
        self.val_loss_epoch = []

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        # if we have labels we are going to have decoder_attenttion_mask which is calculated from the summary encoding and the labels

        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )

        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["article_input_ids"]
        attention_mask = batch["article_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["article_input_ids"]
        attention_mask = batch["article_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )

        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.val_loss.append(loss)
        return loss

    def on_validation_epoch_end(self, outputs):
      avg = sum(self.val_loss)/len(self.val_loss)
      self.val_loss_epoch.append(avg)
      self.val_loss = []
      avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
      self.logger.experiment.add_scalar('loss',avg_loss, self.current_epoch)

    # def test_step(self, batch, batch_idx):
    #     input_ids = batch["article_input_ids"]
    #     attention_mask = batch["article_attention_mask"]
    #     labels = batch["labels"]
    #     labels_attention_mask = batch["labels_attention_mask"]

    #     loss, outputs = self(
    #         input_ids=input_ids,
    #         attention_mask=attention_mask,
    #         decoder_attention_mask=labels_attention_mask,
    #         labels=labels
    #     )

    #     self.log("test_loss", loss, prog_bar=True, logger=True)
    #     return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=0.0001)

In [30]:
model = NewsHeadlineModel()

In [31]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="news-headline")

trainer = pl.Trainer(
    logger=logger,
    callbacks=checkpoint_callback,
    max_epochs=1
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [32]:
trainer.fit(model, data_module)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:369: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 2645: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [33]:
def generate_headline(text):
  text_encoding = tokenizer(
      text,
      max_length=512,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )

  generated_ids = model.model.generate(
      input_ids=text_encoding["input_ids"],
      attention_mask=text_encoding["attention_mask"],
      max_length=150,
      num_beams=2, # beam search
      repetition_penalty=2.5,
      length_penalty=1.0,
      early_stopping=True # To speed up the process
  )

  preds = [
      tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
      for gen_id in generated_ids
  ]

  return "".join(preds)

In [34]:
sample_row = dev_dataset.iloc[0]
text = sample_row["news"]

In [35]:
sample_row["headline"]

'3rd Victim Dead in Quarry Shooting; Manhunt Still On'

In [36]:
generate_headline(text)

'3 Dead, 5 Injured in California Quarry Shooting'

In [37]:
sample_row = dev_dataset.iloc[10]
text = sample_row["news"]

In [38]:
sample_row["headline"]

'Deficit Breaks $1T Again'

In [39]:
generate_headline(text)

'Feds Expect $1.29T Deficit for Second Year in Row'

In [40]:
!pip install rouge

In [41]:
from rouge import Rouge
ROUGE = Rouge()

In [42]:
sample_row = dev_dataset.iloc[1]
text = sample_row["news"]
reference = sample_row["headline"]
candidate = generate_headline(text)

In [43]:
x = ROUGE.get_scores(candidate, reference)
x[0]

{'rouge-1': {'r': 0.3333333333333333,
  'p': 0.3333333333333333,
  'f': 0.3333333283333334},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.3333333333333333,
  'p': 0.3333333333333333,
  'f': 0.3333333283333334}}

In [44]:
res = {}
res['rouge-1'] = {}
res['rouge-2'] = {}
res['rouge-l'] = {}
for key in res:
  res[key]['r'] = 0
  res[key]['p'] = 0
  res[key]['f'] = 0
for index in range(len(dev_dataset.index)):
  sample_row = dev_dataset.iloc[index]
  text = sample_row["news"]
  reference = sample_row["headline"]
  candidate = generate_headline(text)
  score = ROUGE.get_scores(candidate, reference)
  for key in res:
    res[key]['r'] += score[0][key]['r']
    res[key]['p'] += score[0][key]['p']
    res[key]['f'] += score[0][key]['f']
for key in res:
  res[key]['r'] /= len(dev_dataset.index)
  res[key]['p'] /= len(dev_dataset.index)
  res[key]['f'] /= len(dev_dataset.index)
res

{'rouge-1': {'r': 0.3266328279218455,
  'p': 0.3473737033462195,
  'f': 0.3302829161916125},
 'rouge-2': {'r': 0.13207632689133714,
  'p': 0.14062215796888072,
  'f': 0.1331535816929239},
 'rouge-l': {'r': 0.30417998008844843,
  'p': 0.3237290721963027,
  'f': 0.30767337953105495}}